In [59]:
import pandas as pd
import numpy as np
import matplotlib as mpl
from datetime import datetime
import plotly
plotly.tools.set_credentials_file(username='bzjin', api_key='jmaLQFWVMgugFAITZZBC')
import plotly.plotly as py
import plotly.graph_objs as go


In [60]:
# "offender_offenses_ccs.csv" has 414,617 rows x 17 columns
offenses = pd.read_csv("offender_offenses_ccs.csv")
first = offenses.iloc[0:10000,:]
reviews.head()

,DCNumber,Sequence,OffenseDate,DateAdjudicated,County,CaseNumber,prisonterm,ProbationTerm,ParoleTerm,adjudicationcharge_descr,qualifier_descr,adjudication_descr
0,000084,1,01/27/76 00:00:00,10/08/76 00:00:00,DUVAL,7600296.0,9999998,0,0,1ST DG MUR/PREMED. OR ATT.,PRINCIPAL,ADJUDICATION NOT WITHHELD
1,000256,1,01/29/71 00:00:00,10/12/71 00:00:00,LEON,6528.0,9999998,0,9999998,1ST DG MUR/PREMED. OR ATT.,PRINCIPAL,ADJUDICATION NOT WITHHELD
2,000256,2,08/17/90 00:00:00,12/21/90 00:00:00,LEON,9003492.0,70000,0,0,BURGUNOCCSTRUC/CV OR ATT.,PRINCIPAL,ADJUDICATION NOT WITHHELD
3,000316,1,10/15/59 00:00:00,12/04/59 00:00:00,MIAMI-DADE,5905444.0,150000,0,0,ROBBERY,PRINCIPAL,ADJUDICATION NOT WITHHELD
4,000316,2,01/15/62 00:00:00,02/02/62 00:00:00,BROWARD,6236552.0,20000,0,0,BURGUNOCCSTRUC/CV OR ATT.,PRINCIPAL IN ATTEMPT,ADJUDICATION NOT WITHHELD


In [61]:
#Charges
chargeSeries = pd.Series(first["adjudicationcharge_descr"])
chargeArray = np.array(chargeSeries, dtype=pd.Series)

unique, counts = np.unique(chargeArray, return_counts=True)
d = {}
for x in unique:
    d[x] = counts[np.where(unique==x)][0]

#chargedata=[go.Bar(x=unique, y=counts)]
#py.iplot(chargedata, filename='basic-bar')

In [62]:
# Offense classifications from http://www.dc.state.fl.us/OffenderSearch/offensecategory.aspx#ED
classifications = pd.read_csv("offense_classifications.csv", low_memory=False)
classifications.head()

,code,name,category
0,1350,3+ BATTERY,Assault and Battery
1,3840,ABUSE ELDER/DISABLD,Assault and Battery
2,3841,ABUSE ELDER/DISBLD-HARM,Assault and Battery
3,1318,AGG ASSLT-INTENT COMMIT FELONY,Assault and Battery
4,1317,AGG ASSLT-W/WPN NO INTENT TO K,Assault and Battery


In [95]:
#This groups the types of ALL offenses together. An array stores those offenses that have not been accounted for or labelled yet. 
class_dict = {}

cate_Series = pd.Series(classifications["category"])
cate_Array = np.array(cate_Series, dtype=pd.Series)
unique, counts = np.unique(cate_Array, return_counts=True)

names = np.array(classifications["name"], dtype=pd.Series)

for x in unique:
    class_dict[x] = 0

missing_count = []
for x in names:
    index = np.where(names == x)[0][0]
    category = classifications["category"][index]
    if (x in list(d.keys())):
        class_dict[category] = class_dict[category] + d[x]
    else: 
        missing_count.append(x)
    
print("Missing count: ", len(missing_count))
print(missing_count)

Missing count:  975
['ABUSE ELDER/DISBLD-HARM', 'AGG. BATT. WEIGHT/SEC.OFF.', 'AGG.ABUSE - ELDER/DISABLD', 'AGG.ASSAULT/65 YRS./OLDER', 'AGG.FLEE LEO INJ./DAMAGE', 'AGGRAV ASSLT-FAMILY-GUN', 'AGGRAV ASSLT-FAMILY-OTHER WEAP', 'AGGRAV ASSLT-FAMILY-STRONGARM', 'AGGRAV ASSLT-NONFAMILY-OTHER', 'AGGRAV ASSLT-NONFAMILY-STGARM', 'AGGRAV ASSLT-POL OFF-STGARM', 'AGGRAV ASSLT-POLICE OFF-GUN', 'AGGRAV ASSLT-POLICE OFF-OTHER', 'AGGRAV ASSLT-PUB OFF-OTHER', 'AGGRAV ASSLT-PUB OFF-STGARM', 'AGGRAV ASSLT-PUB OFFICIAL-GUN', 'AGGRAV. ASSAULT/BATTERY', 'AGGRAVATED ASSAULT - GUN', 'ARSON,WILLFUL INJ.TO PSN.', 'ASSAULT BY PRISONER', 'ATTEMPT MURDER LAW ENFORCE OFF', 'BAT.AMB.DRIV/EMT/PARAMED', 'BATTERY OF HRS EMPLOYEE', 'BOD.INJ./NOT 782.04(3)OFF.', 'DIRECT PERSON TO DISC. F/A FR.', 'DISCH.DESTR.DEVICE W/HARM', 'DISCH.DESTRUCT.DEV.DAM.', 'DISCH.DESTRUCT.DEV.W/INJ', 'DISCH.WEAP.SCHOOL GROUNDS', 'DISCHARGE DESTRUC.DEVICE', 'EXPLOT.OF AGED PERSON,CAUSE BO', 'HARMS PUBLIC SERVANT', 'IMPRSN.LEO/FELNY-INJ/DETH', 

In [96]:
list_offenses = list(class_dict.keys())
list_counts = list(class_dict.values())

offenses_data=[go.Bar(x=list_offenses, y=list_counts)]
py.iplot(offenses_data, filename='basic-bar')